In [78]:
# Librerias Necesarias.
import pandas as pd # Para trabajar con bases de datos en objeto data_frame
from gurobipy import * # Libreria de Solver Gurobi 
import numpy as np # Libreria para Matrices

T1 = 8/100000 # Defino tasa por habitantes
T2 = 316 # Defino la tasa de delitos 

# Creamos una funcion que tome los parámetros creados
# Este modelo toma los datos de poblacion, demandas y tasas
# Saca una tabla de la organízación de fiscales que debe tener cada canton.

def Modelos_Provincia(T1, T2):
    Poblacion =  pd.read_excel('Poblacion.xlsx', sheet_name= "Hoja1" ) 
    POB = dict() ## Datos de la población de la provincia
    Datos_cantones = pd.read_excel('Canton.xlsx', sheet_name= "Frecuencia") 
    Name = ["FISCALIA DE PERSONAS Y GARANTIAS","FISCALIA DE DELINCUENCIA ORGANIZADA, TRANSNACIONAL E INTERNACIONAL","FISCALIA DE ADMINISTRACION PUBLICA","FISCALIA DE SOLUCIONES RAPIDAS","FISCALIA DE VIOLENCIA DE GENERO","FISCALIA DE PATRIMONIO CIUDADANO","FISCALIA DE FE  PUBLICA","FISCALIA DE ACCIDENTES DE TRANSITO","UNIDAD ANTILAVADO DE ACTIVOS"]
    Cantones = Datos_cantones.iloc[:,1]
    for i in range(len(Poblacion.iloc[:,1])):
        POB[Poblacion.iloc[i,1]] = Poblacion.iloc[i,2]
    Resultados = pd.DataFrame(columns=['Nombre_Fiscalía', 'Número_Fiscales'])
    for i in range(len(Name)):
        Resultados = Resultados.append({'Nombre_Fiscalía': Name[i] , 'Cero':0}, ignore_index=True)
    Res_Pichincha = Resultados
    for c in Cantones:
        print(c)
        M =  pd.read_excel('Canton.xlsx', sheet_name= c) 
        NameF = ["Fiscalia 1","Fiscalia 2","Fiscalia 3","Fiscalia 4","Fiscalia 5","Fiscalia 6","Fiscalia 7","Fiscalia 8","Fiscalia 9"]
        df = pd.DataFrame()
        df['Fiscalía'] = NameF
        df['Nombre'] = Name
        NameGr = ["Grupo1", "Grupo2", "Grupo3"]
        ## Creación de los nodos usando tuplas
        D = tuplelist([]) ## Conjunto de Grupos de delitos
        F = tuplelist([]) ## Nombre de Fiscalías
        F1 = tuplelist([]) ## Numero de Fiscalías
        DXF = tuplelist([]) ## Conjunto de (Grupos de Delitos, Fiscalias)

        #print(sum(M.iloc[:,2:3]))
        ## Creacion de los elementos de las tuplas dependiendo de los datos encontrados en el modelo

        D_df = tupledict({})
        
        for i in range(2,len(M.iloc[1,:])):
            for j in range(len(M)):
                F.append(NameF[j])
                F1.append(Name[j])
                if M.iloc[j,i] >0 :
                    DXF.append((NameGr[i-2],NameF[j]))
                    D_df[(NameGr[i-2],NameF[j])] = round(M.iloc[j,i]) 
        F = list(set(F)) ## Los Nodos de Fiscalias 
            


        ## Creamos el Modelo para Quito
        m = Model('Modelo_FGE')


        ## Creación de Variables
        # Cantidad de delitos asignados a la fiscalía 
        X = m.addVars(DXF, name="Cant_Del", vtype=GRB.INTEGER) 
        # Número de fiscales en cada fiscalía
        Y = m.addVars(F, name="Cant_Fis", vtype=GRB.INTEGER)

        ## PARAMETROS    
        # Demanda de delitos de cada grupo en la fiscalía f


        # Poblacion del cantón Quito
        ## Modelo Original

        #FUNCIÓN OBJETIVO
        m.setObjective(((Y.sum(F))) , GRB.MINIMIZE)

        #RESTRICCIONES
        #Añadir restricciones del modelo.
        #1. Se cumpla la demanda de Delitos por canton 
        for d in DXF:
            m.addConstr(X[d] == D_df[d], "demandadelitos_{}".format(d))

        #2. Se cumpla la carga de delitos por fiscal 
        if  POB[c] >= 31000:
            for f in F:
                m.addConstr(((X.sum(NameGr,f)) <= Y[f]*T2) , "tasa de carga_{}".format(f))
        if  POB[c] < 31000:
            m.addConstr(((X.sum(NameGr,F)) <= Y.sum(F)*T2) , "tasa de carga_{}".format(f))

        m.addConstr(Y.sum(F) >= POB[c]*T1, "Se cumpla tasa T1") # Restricción de fiscales por poblacion

        ## Optimizar modelo

        m.optimize()

        Resultados = pd.DataFrame(columns=['Nombre_Fiscalía', 'Número_Fiscales'])

        Fiscalias = ["FISCALIA DE PERSONAS Y GARANTIAS","FISCALIA DE DELINCUENCIA ORGANIZADA, TRANSNACIONAL E INTERNACIONAL","FISCALIA DE PATRIMONIO CIUDADANO","FISCALIA DE SOLUCIONES RAPIDAS","FISCALIA DE FE  PUBLICA", "FISCALIA DE ADMINISTRACION PUBLICA", "FISCALIA DE VIOLENCIA DE GENERO","FISCALIA DE ACCIDENTES DE TRANSITO","UNIDAD ANTILAVADO DE ACTIVOS" ]
        for f in F:
            for j in range(9):
                if df.iloc[j, 0] == f:
                    Resultados = Resultados.append({'Nombre_Fiscalía': df.iloc[j, 1] , 'Número_Fiscales':Y[f].x}, ignore_index=True)
        suma = 0 # Cantidad de delitos para comparaciones posteriores.
        for d in DXF:
            suma = suma + X[d].x

        Resultados = Resultados.append({'Nombre_Fiscalía': "TOTAL" , 'Número_Fiscales': sum(Resultados.iloc[:, 1])}, ignore_index=True)
        Res_Pichincha = pd.merge(Res_Pichincha,Resultados, on = 'Nombre_Fiscalía')
    
    Res_Pichincha = Res_Pichincha.drop(['Cero'], axis=1)
    columnas = ['Nombre_Fiscalía', 'Ninguno']
    for ca in Cantones:
        columnas.append(ca)
    Res_Pichincha.columns = columnas
    Res_Pichincha = Res_Pichincha.drop(['Ninguno'], axis=1)
    
    return(Res_Pichincha)

# Fiscales por provincia.
Modelos_Provincia( T1, T2)


PASTAZA (PUYO)
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 30 rows, 29 columns and 58 nonzeros
Model fingerprint: 0x726619bf
Variable types: 0 continuous, 29 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+00, 7e+02]
Found heuristic solution: objective 15.0000000
Presolve removed 30 rows and 29 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 15 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.500000000000e+01, best bound 1.500000000000e+01, gap 0.0000%
MERA
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with

,Nombre_Fiscalía,PASTAZA (PUYO),MERA,ARAJUNO,SANTA CLARA
0,FISCALIA DE PERSONAS Y GARANTIAS,1.0,-0.0,-0.0,-0.0
1,"FISCALIA DE DELINCUENCIA ORGANIZADA, TRANSNACI...",1.0,3.0,1.0,1.0
2,FISCALIA DE ADMINISTRACION PUBLICA,1.0,-0.0,-0.0,-0.0
3,FISCALIA DE SOLUCIONES RAPIDAS,5.0,-0.0,-0.0,-0.0
4,FISCALIA DE VIOLENCIA DE GENERO,4.0,-0.0,-0.0,-0.0
5,FISCALIA DE PATRIMONIO CIUDADANO,1.0,-0.0,-0.0,-0.0
6,FISCALIA DE FE PUBLICA,1.0,-0.0,-0.0,-0.0
7,FISCALIA DE ACCIDENTES DE TRANSITO,1.0,-0.0,-0.0,-0.0
8,UNIDAD ANTILAVADO DE ACTIVOS,-0.0,-0.0,-0.0,-0.0
